# Tutorial to read in CANDELS data and plot figures

In [2]:
## set paths to catalog/script files and output directories (for plots)
data_dir = '../hdf5files/'
script_dir = './'
out_dir = './'

In [10]:
## load plotting routines
execfile('/Users/Andy/Documents/SIP18/SIP_jerome_scripts2/plot_routines.py')

## Read in and setup data for plotting

### GOODS-S

In [18]:
## read in catalog for GOODS-S 
gds = Table.read(data_dir+'gds.hdf5')

In [21]:
# define selection cuts for GOODS-S

## F160W < 24.5 mag
## SE flag (PhotFlag) = 0
## CLASS_STAR < 0.9
## redshifts between 0.2 and 2.5
## masses between 9 and 11
## GALFIT flag = 0

gds_selection = np.where( \
                      (gds['mag_f160w_4']<24.5) &\
                      (gds['PhotFlag']==0) & \
                      (gds['CLASS_STAR']<0.9) & \
                      (gds['z_best']>0.5) & (gds['z_best']<2.5) & \
                      (log10(gds['M_med'])>9.0) & (log10(gds['M_med'])<11.0) &\
                      (gds['f_f160w']==0))


KeyError: 'mag_f160w_4'

In [22]:
print gds_selection

NameError: name 'gds_selection' is not defined

In [ ]:
## define new structure with only selected objects
gds_final = gds[gds_selection]

In [ ]:
## define variables for GOODS-S

u_gds = gds_final['restUXbessel']  ## rest-frame U magnitude
v_gds = gds_final['restVbessel']   ## rest-frame V magnitude
j_gds = gds_final['restJpalomar']  ## rest-frame J magnitude

uv_gds = u_gds - v_gds
vj_gds = v_gds - j_gds

uvprime_gds = gds_final['uvrot'] ## (U-V)' color
vjprime_gds = gds_final['vjrot'] ## (V-J)' color

mstar_gds = log10(gds_final['M_med']) ## log (median) stellar mass (in M_sun)
z_gds = gds_final['z_best']  ## redshift

ssfr_gds = gds_final['ssfr_uv_corr'] ## log specific SFR (UV,corr)
sf_flag_gds = gds_final['sf_flag'] ## -1 if quiescent, 1 if star-forming, acc to UVJ
delssfr_gds = gds_final['delta_ssfr'] ## Delta log SSFR
av_gds = gds_final['med_av'] ## (median) visual attenuation (in mag)

sma_gds = log10(gds_final['sma_f160w_kpc']) ## log SMA (in kpc)
delsma_gds = gds_final['delta_sma']  ## Delta log SMA
n_gds = gds_final['n_f160w']  ## Sersic index n
ba_gds = gds_final['q_f160w'] ## axis ratio b/a

### UDS

In [ ]:
uds = Table.read('uds_all.hdf5')

In [ ]:
uds_selection = where( \
                      (uds['mag_f160w_4']<26.7) &\
                      (uds['PhotFlag']==0) & \
                      (uds['CLASS_STAR']<0.9) & \
                      (uds['z_best']>0.2) & (uds['z_best']<2.5) & \
                      (log10(uds['M_med'])>9.0) & (log10(uds['M_med'])<11.0) &\
                      (uds['f_f160w']==0))

In [ ]:
uds_final = uds[uds_selection]

In [ ]:
## define variables for UDS

u_uds = uds_final['restUXbessel']  ## rest-frame U magnitude
v_uds = uds_final['restVbessel']   ## rest-frame V magnitude
j_uds = uds_final['restJpalomar']  ## rest-frame J magnitude

uv_uds = u_uds - v_uds
vj_uds = v_uds - j_uds

uvprime_uds = uds_final['uvrot'] ## (U-V)' color
vjprime_uds = uds_final['vjrot'] ## (V-J)' color

mstar_uds = log10(uds_final['M_med']) ## log (median) stellar mass (in M_sun)
z_uds = uds_final['z_best']  ## redshift

ssfr_uds = uds_final['ssfr_uv_corr'] ## log specific SFR (UV,corr)
sf_flag_uds = uds_final['sf_flag'] ## -1 if quiescent, 1 if star-forming, acc to UVJ
delssfr_uds = uds_final['delta_ssfr'] ## Delta log SSFR
av_uds = uds_final['med_av'] ## (median) visual attenuation (in mag)

sma_uds = log10(uds_final['sma_f160w_kpc']) ## log SMA (in kpc)
delsma_uds = uds_final['delta_sma']  ## Delta log SMA
n_uds = uds_final['n_f160w']  ## Sersic index n
ba_uds = uds_final['q_f160w'] ## axis ratio b/a

### Join GOODS-S and UDS variables together to be able to plot both at the same time

In [ ]:
uv = concatenate([u_gds-v_gds,u_uds-v_uds])
vj = concatenate([v_gds-j_gds,v_uds-j_uds])

uvprime = concatenate([uvprime_gds,uvprime_uds])
vjprime = concatenate([vjprime_gds,vjprime_uds])

mass = concatenate([mstar_gds,mstar_uds])
redshift = concatenate([z_gds,z_uds])

ssfr = concatenate([ssfr_gds,ssfr_uds])
sf_flag = concatenate([sf_flag_gds,sf_flag_uds])
delssfr = concatenate([delssfr_gds,delssfr_uds])
av = concatenate([av_gds,av_uds])

sma = concatenate([sma_gds,sma_uds])
delsma = concatenate([delsma_gds,delsma_uds])
sersic = concatenate([n_gds,n_uds])
ba = concatenate([ba_gds,ba_uds])

## Example plots

### $UVJ$ Diagram for whole sample

In [ ]:
fig=figure(figsize=(8,8))
scatter(vj,uv,c='gray',edgecolors='none',s=8,alpha=0.7)
xlim(-0.25,2.5)
ylim(0,2.2)
xlabel('V-J')
ylabel('U-V')

### $UVJ$ grid diagram

To make a grid of panels (mass and redshift bins), use the plot_grid command

The 4 required arguments are: xdata, ydata, mass, redshift

The remaining arguments are "optional", but important to specify axis ranges, labels, color-coding

In [ ]:
ax = plot_grid(vj,uv,mass,redshift,\
               xmin=-0.25,xmax=2.5,xlabel='V-J',\
               ymin=0,ymax=2.2,ylabel='U-V')

To color-code points, use the zdata argument, and set associated parameters

In [ ]:
ax = plot_grid(vj,uv,mass,redshift,\
               xmin=-0.25,xmax=2.5,xlabel='V-J',\
               ymin=0,ymax=2.2,ylabel='U-V',\
               zdata=ssfr,zmin=-10.7,zmax=-8,zlabel='log SSFR',cmap=nipy_r)

You can also generate binned (pixelated) versions of the diagram by setting binned=True (default is 20 bins along x and y)

Each pixel gives the median value of the z-parameter in that pixel

Optionally, specify the number of bins along the x and y axes by setting nxbins=N and nybins=N

In [ ]:
ax = plot_grid(vj,uv,mass,redshift,\
               xmin=-0.25,xmax=2.5,xlabel='V-J',\
               ymin=0,ymax=2.2,ylabel='U-V',\
               zdata=ssfr,zmin=-10.7,zmax=-8,zlabel='log SSFR',cmap=nipy_r,\
               binned=True)

You can plot data satisfying specific criteria by setting the "condition" argument

When specifying multiple conditions, surround each with parentheses and link with &

In [ ]:
## Only plot galaxies that are "round" (b/a>0.5) and star-forming (sf_flag=1)
ax = plot_grid(vj,uv,mass,redshift,\
               xmin=-0.25,xmax=2.5,xlabel='V-J',\
               ymin=0,ymax=2.2,ylabel='U-V',\
               zdata=ssfr,zmin=-10.7,zmax=-8,zlabel='log SSFR',cmap=nipy_r,\
               condition=((ba>0.5) & (sf_flag==1)) )

### Other test plots

In [ ]:
ax = plot_grid(delsma,delssfr,mass,redshift,\
               xmin=-0.6,xmax=0.6,xlabel='Delta SMA',\
               ymin=-1.5,ymax=1.2,ylabel='Delta SSFR',\
               zdata=av,zmin=0,zmax=2.3,zlabel='A_V',cmap=nipy,\
               condition=((sf_flag==1)) )

## you can plot lines in each panel using add_line
xx=linspace(-1,1,10)
add_line(ax,xx,xx*0-0.4,c='r',ls='--')

In [ ]:
ax = plot_grid(delsma,ba,mass,redshift,\
               xmin=-0.6,xmax=0.6,xlabel='Delta SMA',\
               ymin=0,ymax=1.1,ylabel='Axis ratio b/a',\
               zdata=av,zmin=0,zmax=2.3,zlabel='A_V',cmap=nipy,\
               condition=((sf_flag==1) & (delssfr>-0.4)) )

savefig('/Users/jjfang/Desktop/test.pdf')